## Contracts for an AND logic gate composed of three subsystems using Pacti

Import the libraries:

In [1]:
from pacti.terms.polyhedra import *

Write contracts for subsystem 1 and subsystem 2:

In [2]:
contract1 = {
        "_comment1": "contract for subsystem 1, the assumption is that if input (u_1)"
        "is greater than u_1_min = 1, then it guarantees that the output (x_1) is greater than x_1_min = 1.5",
        "input_vars":[
            "u_1"
        ],
        "output_vars":[
            "x_1"
        ],
        "assumptions":
        [
            {"coefficients":{"u_1":-1},
            "constant":-1}
        ],
        "guarantees":
        [
            {"coefficients":{"x_1":-1},
            "constant":-1.5}
        ]
    }
contract2 = {
        "_comment2": "contract for subsystem 2, the assumption is that if input (u_2)"
        "is greater than u_2_min = 1 then it guarantees that the output (x_2) is greater than x_2_min = 0.3",
        "input_vars":[
            "u_2"
        ],
        "output_vars":[
            "x_2"
        ],
        "assumptions":
        [
            {"coefficients":{"u_2":-1},
            "constant":-1}
        ],
        "guarantees":
        [
            {"coefficients":{"x_2":-1},
            "constant":-0.3}
        ]
    }

Use `Pacti` to compute the composition of contract1 and contract2

In [3]:
c1 = PolyhedralContract.from_dict(contract1)
c2 = PolyhedralContract.from_dict(contract2)

print("Contract1:\n" + str(c1))
print("Contract2:\n" + str(c2))

Contract1:
InVars: [<Var u_1>]
OutVars:[<Var x_1>]
A: -1*u_1 <= -1.0
G: -1*x_1 <= -1.5
Contract2:
InVars: [<Var u_2>]
OutVars:[<Var x_2>]
A: -1*u_2 <= -1.0
G: -1*x_2 <= -0.30000000000000004


## Composition

In [4]:
contract_comp, _ = c1.compose(c2)

In [5]:
print(contract_comp)

InVars: [<Var u_1>, <Var u_2>]
OutVars:[<Var x_1>, <Var x_2>]
A: -1*u_1 <= -1.0, -1*u_2 <= -1.0
G: -1*x_1 <= -1.5, -1*x_2 <= -0.30000000000000004


Post-process `contract_comp` (the composition of contract1 and contract2) (if needed as json)

In [8]:
contract12 = write_contract(contract_comp)

In [9]:
contract12

{'input_vars': ['u_1', 'u_2'],
 'output_vars': ['x_1', 'x_2'],
 'assumptions': [{'constant': -1.0, 'coefficients': {'u_1': -1.0}},
  {'constant': -1.0, 'coefficients': {'u_2': -1.0}}],
 'guarantees': [{'constant': -1.5, 'coefficients': {'x_1': -1.0}},
  {'constant': -0.30000000000000004, 'coefficients': {'x_2': -1.0}}]}

Write contract for third subsystem

In [10]:
contract3 = {
        "_comment3": "contract for subsystem 3, the assumption is that if input (x_1 and x_2) are greater than x_1_min = 1.5 and x_2_min = 0.3 then it guarantees that the output (y) is greater than y_eps = 1.5",
        "input_vars":[
            "x_1", "x_2"
        ],
        "output_vars":[
            "y"
        ],
        "assumptions":
        [
            {"coefficients":{"x_1":-1},
            "constant":-1.5},
            {"coefficients":{"x_2":-1},
            "constant":-0.3}
        ],
        "guarantees":
        [
            {"coefficients":{"y":-1},
            "constant":-1.5}
        ]
    }

In [11]:
c3 = PolyhedralContract.from_dict(contract3)
print("Contract3:\n" + str(c3))

Contract3:
InVars: [<Var x_1>, <Var x_2>]
OutVars:[<Var y>]
A: -1*x_1 <= -1.5, -1*x_2 <= -0.3
G: -1*y <= -1.5


Compose contract12 with contract3:

In [12]:
and_gate_contract, _ = contract_comp.compose(c3)

In [13]:
print(and_gate_contract)

InVars: [<Var u_1>, <Var u_2>]
OutVars:[<Var y>]
A: -1*u_1 <= -1.0, -1*u_2 <= -1.0
G: -1*y <= -1.5


## Quotient

Assume that we have `contract_comp` (the composition of `contract1` and `contract2`) and the `and_gate_contract` available. Synthesize the contract of subsystem 3 using quotient.

In [14]:
synthesized_c3, _ = and_gate_contract.quotient(contract_comp)

In [15]:
print(synthesized_c3)

InVars: [<Var x_1>, <Var x_2>]
OutVars:[<Var y>]
A: -1*x_1 <= -1.5, -1*x_2 <= -0.30000000000000004
G: -1*y <= -1.5


### Other quotient examples

Assume that we have the contracts of subsystem 2 and 3, and the top-level `and_gate_contract` available. To synthesize the contract of subsystem 1, we first compute the composition of subsystems 2 and 3 and then quotient it from the top-level contract.


In [16]:
synthesize_c1, _ = and_gate_contract.quotient(c2.compose(c3))

In [17]:
print(synthesize_c1)

InVars: [<Var u_1>]
OutVars:[<Var x_1>]
A: -1*u_1 <= -1.0
G: -1*x_1 <= -1.5


Assume that we have the contracts of subsystem 1 and 3, and the top-level `and_gate_contract` available. To synthesize the contract of subsystem 2, we first compute the composition of subsystems 1 and 3 and then quotient it from the top-level contract.


In [18]:
synthesize_c2 = and_gate_contract.quotient(c1.compose(c3))

In [19]:
print(synthesize_c2)

InVars: [<Var u_2>]
OutVars:[<Var x_2>]
A: -1*u_2 <= -1.0
G: -1*x_2 <= -0.30000000000000004


Assume that we have the contracts of subsystem 1 and the top-level `and_gate_contract` available. To synthesize the contracts of all other subsystems, we first compute the quotient of subsystem 3 from the top-level contract. This contract may be further decomposed if there are other subsystem contracts available.

In [20]:
synthesize_c12, _ = and_gate_contract.quotient(c3)

In [21]:
print(synthesize_c12)

InVars: [<Var u_1>, <Var u_2>]
OutVars:[<Var x_1>, <Var x_2>]
A: -1*u_1 <= -1.0, -1*u_2 <= -1.0
G: -1*x_1 <= -1.5, -1*x_2 <= -0.30000000000000004


In [22]:
synthesize_c1, _ = synthesize_c12.quotient(c2)

In [23]:
print(synthesize_c1)

InVars: [<Var u_1>]
OutVars:[<Var x_1>]
A: -1*u_1 <= -1.0
G: -1*x_1 <= -1.5


In [24]:
print('Success')

Success
